# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

import warnings
#warnings.filterwarnings("ignore")

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,287.02,76,85,2.78,SH,1730754836
1,1,ushuaia,-54.8000,-68.3000,282.06,66,51,1.56,AR,1730754726
2,2,mikhaylov,54.2292,39.0186,269.39,94,38,5.97,RU,1730754838
3,3,adamstown,-25.0660,-130.1015,294.45,80,95,8.28,PN,1730754839
4,4,isafjordur,66.0755,-23.1240,275.52,83,100,2.12,IS,1730754841


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 800,
    frame_height = 600,
    alpha = 0.5
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_new = city_data_df[(city_data_df['Max Temp'] >= 294) & (city_data_df['Max Temp'] <= 300) & (city_data_df['Humidity'] > 32) & (city_data_df['Cloudiness'] == 0) & (city_data_df['Wind Speed'] < 4.5)  ].copy()
# Drop any rows with null values
city_data_df_new.dropna(inplace=True)

# Display sample data
city_data_df_new

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,35,karachi,24.9056,67.0822,298.05,83,0,2.06,PK,1730754689
36,36,laguna,38.4210,-121.4238,294.29,39,0,2.57,US,1730754882
106,106,nahiyat ash shinafiyah,31.5874,44.6602,295.31,50,0,3.58,IQ,1730754969
140,140,fort bragg,35.1390,-79.0060,296.43,61,0,3.60,US,1730754770
150,150,kununurra,-15.7667,128.7333,299.14,73,0,2.06,AU,1730754985
222,222,maxixe,-23.8597,35.3472,296.03,85,0,3.79,MZ,1730755126
224,224,al wajh,26.2455,36.4525,296.12,49,0,2.06,SA,1730755128
281,281,saint-pierre,-21.3393,55.4781,294.97,78,0,1.54,RE,1730755215
293,293,phaphund,26.6000,79.4667,295.72,35,0,1.73,IN,1730755230
347,347,porto novo,6.4965,2.6036,299.77,78,0,2.28,BJ,1730755302


### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_new[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\NECEE\AppData\Local\Temp\ipykernel_12420\2264778735.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
35,karachi,PK,24.9056,67.0822,83,
36,laguna,US,38.4210,-121.4238,39,
106,nahiyat ash shinafiyah,IQ,31.5874,44.6602,50,
140,fort bragg,US,35.1390,-79.0060,61,
150,kununurra,AU,-15.7667,128.7333,73,
222,maxixe,MZ,-23.8597,35.3472,85,
224,al wajh,SA,26.2455,36.4525,49,
281,saint-pierre,RE,-21.3393,55.4781,78,
293,phaphund,IN,26.6000,79.4667,35,
347,porto novo,BJ,6.4965,2.6036,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "limit":10,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
karachi - nearest hotel: Quetta Agha Hotel
laguna - nearest hotel: Holiday Inn Express & Suites
nahiyat ash shinafiyah - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
kununurra - nearest hotel: No hotel found
maxixe - nearest hotel: Hotel Herry
al wajh - nearest hotel: فندق رالين 2
saint-pierre - nearest hotel: Tropic Hotel
phaphund - nearest hotel: No hotel found
porto novo - nearest hotel: Hôtel 6500 CFA
toliara - nearest hotel: Ambary
inhambane - nearest hotel: Hotel de Inhambane
darab - nearest hotel: محدوده هتل جهانگردی
ayden - nearest hotel: Country Inn & Suites, Greenville, NC
chiredzi - nearest hotel: Nesbitt Arms Hotel
cambria - nearest hotel: Olallieberry Inn
mulayjah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
35,karachi,PK,24.9056,67.0822,83,Quetta Agha Hotel
36,laguna,US,38.4210,-121.4238,39,Holiday Inn Express & Suites
106,nahiyat ash shinafiyah,IQ,31.5874,44.6602,50,No hotel found
140,fort bragg,US,35.1390,-79.0060,61,Airborne Inn Lodging
150,kununurra,AU,-15.7667,128.7333,73,No hotel found
222,maxixe,MZ,-23.8597,35.3472,85,Hotel Herry
224,al wajh,SA,26.2455,36.4525,49,فندق رالين 2
281,saint-pierre,RE,-21.3393,55.4781,78,Tropic Hotel
293,phaphund,IN,26.6000,79.4667,35,No hotel found
347,porto novo,BJ,6.4965,2.6036,78,Hôtel 6500 CFA


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
map_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 700,
    frame_height = 400,
    alpha = 0.5,
    hover_cols=["Lng", "lat", "City", "Humidity" "Hotel Name","Country"]
)

# Display the map
map_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)